# Calibración de IPP

En este cuaderno vamos a preparar los datos para IPP, y a correr el algoritmo de calibración. Al final obtendremos una tabla con los parámetros de cada indicador así como las métricas de bondad de ajuste correspondientes. Después, verificaremos que los parámetros obtenidos son consistentes con los que fueron calculados para el reporte (para los cuales se empleó un criterio de precisión más exigente que el de este cuaderno).

## Preparación de indicadores